In [ ]:
import requests
import time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import os

# (Opsiyonel) .env dosyasını yükle
# from dotenv import load_dotenv
# load_dotenv()

# InfluxDB bağlantı bilgileri
url = "http://localhost:8086"
token = "uIDz4VhQ9tosSxViLxwwgDwo6jF3CsqGfJdK0XeghqCSAqS97c934ddUCybwoBUCBnL1CzElikLwNWap3LAP7g=="
# token = "t33br8eEj6H5bYOgz0ddqR_B8BZp0xjl8x--QCsPjG9-XZaFZBWE5Tp50W58gu9ytlFp725zk_QxeZBRphNa4w=="
org = "Yildiz Technical Univercity"  # <-- Arayüzde yazan birebir haliyle olmalı
bucket = "crypto_bucket_db"       # <-- bucket gerçekten var mı?

# Client başlat
client = InfluxDBClient(url=url, token=token, org=org)
write_api = client.write_api(write_options=SYNCHRONOUS)

# Binance API'den veri çek
def get_binance_price(symbol="BTCUSDT"):
    endpoint = f"https://api.binance.com/api/v3/ticker/price?symbol={symbol}"
    response = requests.get(endpoint)
    data = response.json()
    return float(data["price"])

# Her 5 saniyede bir veri yaz
try:
    while True:
        price = get_binance_price()
        point = (
            Point("crypto_price")
            .tag("symbol", "BTCUSDT")
            .field("price", price)
            .time(time.time_ns(), WritePrecision.NS)
        )
        write_api.write(bucket=bucket, org=org, record=point)
        # print(f"Yazıldı: BTC/USDT = {price}")
        time.sleep(5)

except KeyboardInterrupt:
    print("Veri aktarımı durduruldu.")
finally:
    client.close()
